<a href="https://colab.research.google.com/github/saianusha10/Hands_on_14/blob/main/Hands_on_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Q1.  Dijkstra's algorithm

In [8]:
import heapq


def dijkstra_algorithm(network, source):
    # Priority queue to manage (cost, vertex)
    priority_queue = []
    heapq.heappush(priority_queue, (0, source))

    # Dictionary to hold the shortest distance to each vertex
    shortest_distances = {vertex: float('inf') for vertex in network}
    shortest_distances[source] = 0

    # Dictionary to keep track of the path taken
    path_predecessors = {vertex: None for vertex in network}

    while priority_queue:
        current_cost, current_vertex = heapq.heappop(priority_queue)

        # Skip processing if the current cost is greater than the recorded shortest distance
        if current_cost > shortest_distances[current_vertex]:
            continue

        # Check each neighboring vertex
        for neighbor, edge_cost in network[current_vertex]:
            new_cost = current_cost + edge_cost

            # Update if a shorter distance to the neighbor is found
            if new_cost < shortest_distances[neighbor]:
                shortest_distances[neighbor] = new_cost
                path_predecessors[neighbor] = current_vertex
                heapq.heappush(priority_queue, (new_cost, neighbor))

    return shortest_distances, path_predecessors


# Example network definition
network = {
    'S': [('T', 3), ('Y', 5)],
    'T': [('Y', 2), ('X', 6)],
    'Y': [('T', 1), ('X', 4), ('Z', 6)],
    'X': [('Z', 2)],
    'Z': [('S', 3), ('X', 7)]
}

# Run Dijkstra's algorithm starting from the source vertex 'A'
shortest_distances, path_predecessors = dijkstra_algorithm(network, 'S')

# Print the shortest distances
print("Shortest distances from source 'S':")
for vertex, distance in shortest_distances.items():
    print(f"{vertex}: {distance}")

# Print the paths
print("\nPaths from source 'S':")
for vertex in path_predecessors:
    path = []
    current_vertex = vertex
    while current_vertex is not None:
        path.insert(0, current_vertex)
        current_vertex = path_predecessors[current_vertex]
    print(f"Path to {vertex}: {' -> '.join(path)}")

Shortest distances from source 'S':
S: 0
T: 3
Y: 5
X: 9
Z: 11

Paths from source 'S':
Path to S: S
Path to T: S -> T
Path to Y: S -> Y
Path to X: S -> T -> X
Path to Z: S -> Y -> Z


Q2. Bellman-Ford algorithm

In [9]:
def bellman_ford_algorithm(network, source):
    # Initialize all nodes with infinite distances except the source, which is 0
    min_distances = {vertex: float('inf') for vertex in network}
    min_distances[source] = 0

    # Dictionary to track the predecessor of each vertex in the shortest path
    predecessors = {vertex: None for vertex in network}

    # Extract all edges as a list of (start, end, weight)
    edge_list = [(start, end, cost) for start in network for end, cost in network[start]]

    # Perform |V| - 1 iterations to relax all edges
    for _ in range(len(network) - 1):
        for start, end, cost in edge_list:
            if min_distances[start] + cost < min_distances[end]:
                min_distances[end] = min_distances[start] + cost
                predecessors[end] = start

    # Check for negative weight cycles
    for start, end, cost in edge_list:
        if min_distances[start] + cost < min_distances[end]:
            raise ValueError("The graph contains a negative weight cycle")

    return min_distances, predecessors

# Example graph structure
network = {
    'S': [('T', 3), ('Y', 5)],
    'T': [('Y', 2), ('X', 6)],
    'Y': [('T', 1), ('X', 4), ('Z', 6)],
    'X': [('Z', 2)],
    'Z': [('S', 3), ('X', 7)]
}

# Run the Bellman-Ford algorithm from the source node 'A'
try:
    shortest_distances, path_predecessors = bellman_ford_algorithm(network, 'S')

    # Print the results
    print("Minimum distances from source 'S':")
    for vertex, distance in shortest_distances.items():
        print(f"{vertex}: {distance}")

    print("\nShortest paths from source 'S':")
    for vertex in path_predecessors:
        path = []
        current_vertex = vertex
        while current_vertex is not None:
            path.insert(0, current_vertex)
            current_vertex = path_predecessors[current_vertex]
        print(f"Path to {vertex}: {' -> '.join(path)}")
except ValueError as error:
    print(error)

Minimum distances from source 'S':
S: 0
T: 3
Y: 5
X: 9
Z: 11

Shortest paths from source 'S':
Path to S: S
Path to T: S -> T
Path to Y: S -> Y
Path to X: S -> T -> X
Path to Z: S -> Y -> Z


Q3. Floyd-Warshall algorithm

In [10]:
def floyd_warshall_algorithm(network):
    # Extract all nodes from the graph
    vertices = list(network.keys())
    vertex_indices = {vertex: index for index, vertex in enumerate(vertices)}
    num_vertices = len(vertices)

    # Initialize matrices for shortest paths and next vertices
    min_distances = [[float('inf')] * num_vertices for _ in range(num_vertices)]
    next_vertex = [[None] * num_vertices for _ in range(num_vertices)]

    # Set distance from each vertex to itself as zero
    for idx in range(num_vertices):
        min_distances[idx][idx] = 0

    # Populate initial distances based on edges in the graph
    for origin in network:
        for destination, cost in network[origin]:
            i, j = vertex_indices[origin], vertex_indices[destination]
            min_distances[i][j] = cost
            next_vertex[i][j] = destination

    # Perform the Floyd-Warshall algorithm
    for intermediary in range(num_vertices):
        for source in range(num_vertices):
            for target in range(num_vertices):
                if min_distances[source][intermediary] + min_distances[intermediary][target] < min_distances[source][target]:
                    min_distances[source][target] = min_distances[source][intermediary] + min_distances[intermediary][target]
                    next_vertex[source][target] = next_vertex[source][intermediary]

    return min_distances, next_vertex, vertices


# Function to display the matrix in a readable format
def display_matrix(matrix, vertices):
    print("    ", "     ".join(vertices))
    for i, row in enumerate(matrix):
        print(f"{vertices[i]:<4}", "  ".join(f"{val if val != float('inf') else '∞':<4}" for val in row))


# Graph representation as an adjacency list
network = {
    'S': [('T', 3), ('Y', 5)],
    'T': [('Y', 2), ('X', 6)],
    'Y': [('T', 1), ('X', 4), ('Z', 6)],
    'X': [('Z', 2)],
    'Z': [('S', 3), ('X', 7)]
}

# Execute the Floyd-Warshall algorithm
min_distances, next_vertex, vertices = floyd_warshall_algorithm(network)

# Print the shortest path distance matrix
print("Shortest Path Distance Matrix:")
display_matrix(min_distances, vertices)

Shortest Path Distance Matrix:
     S     T     Y     X     Z
S    0     3     5     9     11  
T    11    0     2     6     8   
Y    9     1     0     4     6   
X    5     8     10    0     2   
Z    3     6     8     7     0   
